# Dataset link -
##### https://data.mendeley.com/datasets/748f8jkphb/3

# Testing if GPU active

In [ ]:
import tensorflow as tf

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    except RuntimeError as e:
        print(e)


In [ ]:
! nvidia-smi

# Image Augmentation for Balancing Data

In [ ]:
import os
import shutil
import random
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import load_img, img_to_array, array_to_img
from tqdm import tqdm

In [ ]:
input_data_dir = 'Dataset\Plants'
output_data_dir = 'Dataset\Plants'
target_num_images = 200

In [ ]:
def augment_and_save_images(class_folder, image_names):
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    output_class_path = os.path.join(output_data_dir, class_folder)
    os.makedirs(output_class_path, exist_ok=True)

    num_images_to_generate = target_num_images - len(image_names)

    if num_images_to_generate > 0:
        for i in tqdm(range(num_images_to_generate), desc=f'Augmenting {class_folder}'):
            random_image_name = random.choice(image_names)
            image_path = os.path.join(input_data_dir, class_folder, random_image_name)

            img = load_img(image_path)
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)

            for batch in datagen.flow(x, batch_size=1):
                augmented_img = array_to_img(batch[0])
                augmented_image_name = f"augmented_{random.randint(1, 100000)}.jpg"
                output_path = os.path.join(output_class_path, augmented_image_name)
                augmented_img.save(output_path)
                break

In [ ]:
for class_folder in os.listdir(input_data_dir):
    class_path = os.path.join(input_data_dir, class_folder)
    existing_images = [image_name for image_name in os.listdir(class_path) if image_name.endswith('.jpg')]
    augment_and_save_images(class_folder, existing_images)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
root_directory = 'Dataset/Plants'

class_names = []
image_counts = []

for class_folder in os.listdir(root_directory):
    class_path = os.path.join(root_directory, class_folder)
    if os.path.isdir(class_path):
        class_names.append(class_folder)
        image_counts.append(len(os.listdir(class_path)))


plt.figure(figsize=(12, 6))
plt.bar(class_names, image_counts)
plt.xlabel('Plant Class')
plt.ylabel('Number of Images')
plt.title('Number of Images in Each Plant Class')
plt.xticks(rotation=90)
plt.tight_layout()

plt.show()

# Splitting to Test and Train data

In [ ]:
output_directory_test_train = 'Dataset/'
os.makedirs(os.path.join(output_directory_test_train, 'Train'), exist_ok=True)
os.makedirs(os.path.join(output_directory_test_train, 'Test'), exist_ok=True)

In [ ]:
os.listdir(output_directory_test_train)
train_ratio = 0.8

In [ ]:
for class_folder in os.listdir(input_data_dir):
    class_path = os.path.join(input_data_dir, class_folder)
    if os.path.isdir(class_path):
        image_files = os.listdir(class_path)
        num_images = len(image_files)
        num_train = int(train_ratio * num_images)

        train_class_dir = os.path.join(output_directory_test_train, 'Train', class_folder)
        test_class_dir = os.path.join(output_directory_test_train, 'Test', class_folder)
        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(test_class_dir, exist_ok=True)

        for i, image_file in enumerate(tqdm(image_files, desc=f'Copying {class_folder}')):
            source_path = os.path.join(class_path, image_file)
            if i < num_train:
                destination_path = os.path.join(train_class_dir, image_file)
            else:
                destination_path = os.path.join(test_class_dir, image_file)
            shutil.copyfile(source_path, destination_path)

# Checking, Resizing, Vectorizing Images

In [ ]:
import numpy as np
from sklearn.utils import shuffle
import cv2

In [ ]:
dataset_directory = 'Dataset/Plants/'

In [ ]:
image_class_name = [image_class for image_class in os.listdir(dataset_directory)]
image_class_name_label = {image_class:index for index, image_class in enumerate(image_class_name)}
total_class = len(image_class_name)
IMAGE_SIZE = (224,224)

In [ ]:
def pre_process(img_path):
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, IMAGE_SIZE) 
    return image

In [ ]:
def load_data():
    
    datasets = ['Dataset/Train/', 'Dataset/Test/']
    output = []
    
    for dataset in datasets:
        
        images = []
        labels = []
        
        print(f"Loading {dataset}")
        
        for folder in os.listdir(dataset):
            label = image_class_name_label[folder]
            
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                img_path = os.path.join(os.path.join(dataset, folder), file)
                image = pre_process(img_path) 
                
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [ ]:
(train_images, train_labels), (test_images, test_labels) = load_data()

In [ ]:
train_images, train_labels = shuffle(train_images, train_labels, random_state=42)

In [ ]:
number_train = train_labels.shape[0]
number_test = test_labels.shape[0]

print (f"Number of training examples: {number_train}")
print (f"Number of testing examples: {number_test}")
print (f"Each image is of size: {IMAGE_SIZE}")

In [ ]:
train_images = train_images / 255.0 
test_images = test_images / 255.0

In [ ]:
train_images

In [ ]:
test_images

# Displaying Sample Images

In [ ]:
def display_examples(image_class_name, images, labels):
    fig = plt.figure(figsize=(10,10))
    fig.suptitle("Some examples of images of the dataset", fontsize=16)
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap=plt.cm.binary)
        plt.xlabel(image_class_name[labels[i]])
    plt.show()

In [ ]:
display_examples(image_class_name, train_images, train_labels)

# Model Import and Configuration

In [ ]:
from keras.applications.vgg16 import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
from keras.models import Model
from keras.layers import Flatten, Dense

custom_top_model = base_model.output
custom_top_model = Flatten()(custom_top_model)
custom_top_model = Dense(256, activation='relu')(custom_top_model)
custom_top_model = Dense(40, activation='softmax')(custom_top_model)

model = Model(inputs=base_model.input, outputs=custom_top_model)

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import Adam

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(),  
              metrics=['accuracy'])


In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(train_images, train_labels, epochs=100, batch_size=2, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_accuracy}')
